## Intro

This notebook pulls out the key parts of the Eikon API tutorial in order to build up useful scripts in the future. This includes any modifications to the files in the tutorial, as well as code snippets and workflows.

## Setting up Eikon API

In each new environment, you will need to reinstall eikon, as well as then import it.

In [ ]:
## "eikon" is an environment create already within anaconda, but it could be something different.

conda activate eikon
pip install eikon # no need to do this if eikon already installed in the environment
import eikon as ek

# plus import any other packages needed
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp

You need to ensure that when you login to eikon, you do so without saving your key in the global environment or anywhere in your code/markdown - doing so will breach your eikon licencing agreement as this API keys are very sensitive information. Once you've saved this, you can then log in to the API.

In [ ]:
## set app_id in your local environment OR via a text file saved in the current working directory

ek.set_app_key(app_id)

## then you're in